### To do:
1. Create DocEmbSim module to get similarity based on a pre-trained SPECTER model file
2. Think about a better way to represent nagative examples
3. Design a feedback mechanisim -- the result of this feedback is translated to some 'connection weight' to an applicaiton-pair. 

### Note: current connection weight sacle:
    1. Non-related ('negative connection')
    4. Destination application is rejected because target
    5. Target application is rejected because destination 

In [15]:
from allennlp.models.archival import load_archive
from allennlp.common.util import import_submodules

import_submodules('specter')

archive_file = "../model.tar.gz"
cuda_device = -1

metadata = '../data/my_training/test.txt'
included_text_fields = 'abstract title'
vocab_dir = './data/vocab/'


overrides = f"{{'model':{{'predict_mode':'true','include_venue':'false'}},\
                'dataset_reader' : {{ 'type':'specter_data_reader','predict_mode':'true',\
                                     'paper_features_path':'{metadata}',\
                                     'included_text_fields': '{included_text_fields}'}},\
                                      'vocabulary' : {{'directory_path':'{vocab_dir}'}}\
                }}"

archive = load_archive(archive_file, cuda_device = cuda_device)
                      
Predictor.from_archive(archive)

ImportError: cannot import name 'import_submodules' from 'allennlp.common.util' (/usr/local/lib/python3.8/site-packages/allennlp/common/util.py)

In [6]:
from allennlp.predictors import Predictor
predictor = Predictor.from_path("../model.tar.gz")
results = predictor.predict(sentence="Did Uriah honestly think he could beat The Legend of Zelda in under three hours?")
for word, tag in zip(results["words"], results["tags"]):
    print(f"{word}\t{tag}")

INFO:transformers.file_utils:PyTorch version 1.5.1 available.
INFO:transformers.file_utils:TensorFlow version 2.2.0 available.
INFO:allennlp.models.archival:loading archive file ../model.tar.gz
INFO:allennlp.models.archival:extracting archive file ../model.tar.gz to temp dir /var/folders/3n/c1thdb3136qc3c_4t5br9r6c0000gq/T/tmptxr2up5x


ConfigurationError: specter is not a registered name for Model. You probably need to use the --include-package flag to load your custom code. Alternatively, you can specify your choices using fully-qualified paths, e.g. {"model": "my_module.models.MyModel"} in which case they will be automatically imported correctly.

In [ ]:
Class trained_model(object) :
    """
        Load trained SPECTER model for embedding.

        Args:
        -----
        model_file       path to the file containing the trained model
    """
    def __init__(model_file) :
        


In [ ]:
model_file = '../model.tar.gz'

In [1]:
import subprocess
import argparse
import logging
logging.basicConfig(level=logging.INFO)

def embed(ids, model, metadata, output_file, cuda_device=0, batch_size=1, vocab_dir='data/vocab',
          included_text_fields = 'abstract title', weights_file=None
          ):
    
    overrides = f"{{'model':{{'predict_mode':'true','include_venue':'false'}},'dataset_reader':{{'type':'specter_data_reader','predict_mode':'true','paper_features_path':'{metadata}','included_text_fields': '{included_text_fields}'}},'vocabulary':{{'directory_path':'{vocab_dir}'}}}}"

    command = [
        'python3',
        'specter/predict_command.py',
        'predict',
        model,
        ids,
        '--include-package',
        'specter',
        '--predictor',
        'specter_predictor',
        '--overrides',
        f'"{overrides}"',
        '--cuda-device',
        str(cuda_device),
        '--output-file',
        output_file,
        '--batch-size',
        str(batch_size),
        '--silent'
    ]
    if weights_file is not None:
        command.extend(['--weights-file', args.weights_file])
    
    logging.info('running command:')
    logging.info(' '.join(command))
    subprocess.run(' '.join(command), shell=True)

In [5]:
import sys
sys.path.append("../")
embed(ids = './data/my_training/test.txt', metadata='./data/my_training/metadata.json',
      model='./model.tar.gz', output_file = './output.jsonl', vocab_dir='./data/vocab', batch_size=16,
      cuda_device=-1)

INFO:root:running command:
INFO:root:python3 specter/predict_command.py predict ./model.tar.gz ./data/my_training/test.txt --include-package specter --predictor specter_predictor --overrides "{'model':{'predict_mode':'true','include_venue':'false'},'dataset_reader':{'type':'specter_data_reader','predict_mode':'true','paper_features_path':'./data/my_training/metadata.json','included_text_fields': 'abstract title'},'vocabulary':{'directory_path':'./data/vocab'}}" --cuda-device -1 --output-file ./output.jsonl --batch-size 16 --silent


In [10]:
!ls '../'

LICENSE            dist               model-output       specter
README.md          env                model.tar.gz       specter.egg-info
archive.tar.gz     experiment_configs requirements.txt   train.sh
build              files.sh           scripts
data               init.sh            setup.py


In [14]:
!pwd

/Users/kipnisal/specter/scripts
